# Construção da Master de Treino
Compreendendo melhor as informações disponíveis, a volumetria e a distribuição dos dados, utilizaremos um histórico de 3 meses do cliente para prever a possibilidade de churn nos 3 meses seguintes.

Para isso, é essencial construir a master de treino e realizar ajustes necessários de feature engineering. Essa etapa incluirá a agregação de dados relevantes, a criação de variáveis preditivas e a aplicação de transformações adequadas nas variáveis existentes, a fim de melhorar a capacidade do modelo de identificar padrões que indiquem a evasão de clientes.

In [ ]:
import pandas as pd

# Lista das safras que serão utilizadas
safras = ['201612', '201609', '201608', '201607']  # Últimos 3 meses

# Inicializando uma lista para armazenar as tabelas unidas
all_join_tables = []

for safra in safras:
    # Filtrando os dados para a safra atual
    transactions_pd_target_full = transactions_pd[transactions_pd['safra'] <= int(safra)].copy()
    transactions_pd_train = transactions_pd[transactions_pd['safra'] < int(safra)].copy()

    # Selecionando a última entrada de cada cliente na tabela de transações
    transactions_pd_train['rownum'] = transactions_pd_train.sort_values(by=['safra', 'msno'], ascending=False)\
        .groupby('msno', sort=False).cumcount().add(1)
    transactions_pd_date = transactions_pd_train[transactions_pd_train['rownum'] == 1].drop('rownum', axis=1)

    transactions_pd_target_full['rownum'] = transactions_pd_target_full.sort_values(by=['safra', 'msno'], ascending=False)\
        .groupby('msno', sort=False).cumcount().add(1)
    transactions_pd_target = transactions_pd_target_full[transactions_pd_target_full['rownum'] == 1].drop('rownum', axis=1)

    # Filtrando logs para a safra atual e os meses anteriores
    members_pd_date = members_pd[members_pd['safra'] == safra]
    logs_pd_date_target = logs_pd[logs_pd['safra'] == int(safra)]
    logs_pd_date_3 = logs_pd[logs_pd['safra'] == int(safra) - 1]
    logs_pd_date_2 = logs_pd[logs_pd['safra'] == int(safra) - 2]
    logs_pd_date_1 = logs_pd[logs_pd['safra'] == int(safra) - 3]

    # Filtrando apenas clientes que ouviram música ou pagaram fatura
    join_tables_target = (members_pd_date.set_index('msno')
                          .join(logs_pd_date_target.set_index('msno'), how='inner', rsuffix='_logs_target')
                          .join(logs_pd_date_3.set_index('msno'), how='inner', rsuffix='_logs_m1')
                          .join(logs_pd_date_2.set_index('msno'), how='inner', rsuffix='_logs_m2')
                          .join(logs_pd_date_1.set_index('msno'), how='inner', rsuffix='_logs_m3')
                          .join(transactions_pd_target.set_index('msno'), how='inner', rsuffix='_transactions_target')
                          .join(transactions_pd_date.set_index('msno'), how='inner', rsuffix='_transactions_m1'))

    # Adicionando a tabela atual à lista de resultados
    all_join_tables.append(join_tables_target)

# Concatenando todas as tabelas unidas em um único DataFrame
join_tables = pd.concat(all_join_tables)

In [ ]:
# Visualizando as primeiras linhas da tabela unida
join_tables.head()

In [ ]:
# Convertendo as colunas de data para datetime
join_tables.registration_init_time = pd.to_datetime(join_tables.registration_init_time, format='%Y%m%d', errors='coerce')
join_tables.safra = pd.to_datetime(join_tables.safra, format='%Y%m', errors='coerce')

# Calculando o tempo desde o registro
join_tables['registration_ate_hoje'] = (join_tables.registration_init_time.fillna(0).astype(int) - 
                                          join_tables.safra.fillna(0).astype(int))
join_tables['registration_ate_hoje'] = np.where(join_tables['registration_ate_hoje'] < 0, 0, join_tables['registration_ate_hoje'])


In [ ]:
# Corrigindo colunas de dias até expirar as transações
joiddn_tables['transaction_date_ate_expire_transactions_m1'] = np.where(join_tables['transaction_date_ate_expire_transactions_m1'] < 0, 0, join_tables['transaction_date_ate_expire_transactions_m1'])
join_tables['transaction_date_ate_expire'] = np.where(join_tables['transaction_date_ate_expire'] < 0, 0, join_tables['transaction_date_ate_expire'])


In [ ]:
# Soma dos segundos ouvidos nos últimos meses
join_tables['total_secs_ult_meses'] = (join_tables.total_secs + 
                                        join_tables.total_secs_logs_m1 + 
                                        join_tables.total_secs_logs_m2 + 
                                        join_tables.total_secs_logs_m3)

# Razão de audições
join_tables['razao_secs_ult_mes'] = join_tables.total_secs / join_tables.total_secs_logs_m1
join_tables['razao_secs_ult_3_mes'] = join_tables.total_secs / (join_tables.total_secs_logs_m1 + 
                                                               join_tables.total_secs + 
                                                               join_tables.total_secs_logs_m2)


In [ ]:
# Visualizar os resultados
print(join_tables.head())